### Victor Rene Pérez Mayen.
### 19005180

### Proyecto Statistical Learning II

#### Parte 3: Red Neuronal Recurrente

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
df = pd.read_csv("crypto_data/LTC-USD.csv", names = ["time", "low", "high", "open", "close", "volume"])

In [3]:
df

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978
5,1528968960,96.459999,96.519997,96.459999,96.519997,16.991997
6,1528969020,96.379997,96.529999,96.510002,96.440002,95.524078
7,1528969080,96.349998,96.529999,96.480003,96.470001,175.205307
8,1528969140,96.349998,96.550003,96.480003,96.400002,43.652802
9,1528969200,96.400002,96.430000,96.430000,96.400002,8.160000


In [4]:
#main_df = pd.DataFrame()

In [5]:
#ratios = ["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]

In [6]:
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    
    df = pd.read_csv(dataset, names = ["time", "low", "high", "open", "close", "volume"])
    df.rename(columns = {"close": f"{ratio}"}, "low", "high", "open", "close", "volume")

BTC-USD


FileNotFoundError: [Errno 2] File b'training_datas/BTC-USD.csv' does not exist: b'training_datas/BTC-USD.csv'

In [7]:
main_df = pd.DataFrame() 

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"] 
for ratio in ratios: 
    print(ratio)
    dataset = f'crypto_data/{ratio}.csv'  
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  

    # renombrar volumen y cerrar para incluir el símbolo de modo que todavía podamos saber qué cierre/volumen es cuál:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True) 
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  

    if len(main_df)==0:  
        main_df = df  
    else: 
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  
main_df.dropna(inplace=True)
print(main_df.head())  

BTC-USD
LTC-USD
BCH-USD
ETH-USD
            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
1528968960    6480.000000        1.490900      96.519997       16.991997   

            BCH-USD_close  BCH-USD_volume  ETH-USD_close  ETH-USD_volume  
time                                                                      
1528968720     870.859985       26.856577      486.01001       26.019083  
1528968780     870.099976        1.124300      486.00000        8.449400  
1528968840     870.789978        1.749862      485.75000       26.994646  
1528968900     870.000000        1.680500      486.00000    

In [8]:
SEQ_LEN = 60  #cuánto tiempo de una secuencia previa para recolectar para RNN
FUTURE_PERIOD_PREDICT = 3  #qué tan lejos en el futuro estamos tratando de predecir
RATIO_TO_PREDICT = "LTC-USD"

In [9]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [10]:
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

In [11]:
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

In [12]:
print(main_df.head())

            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
1528968960    6480.000000        1.490900      96.519997       16.991997   

            BCH-USD_close  BCH-USD_volume  ETH-USD_close  ETH-USD_volume  \
time                                                                       
1528968720     870.859985       26.856577      486.01001       26.019083   
1528968780     870.099976        1.124300      486.00000        8.449400   
1528968840     870.789978        1.749862      485.75000       26.994646   
1528968900     870.000000        1.680500      486.00000       77.355759   
1528968960 